In [1]:
from datasets import load_dataset

In [5]:
data = load_dataset(
  "json",
  data_files="./vividbot/notebooks/vivid_instruct_65k/data/vivid_instruct_65k_unprocessed.jsonl",
)

Generating train split: 0 examples [00:00, ? examples/s]

In [8]:
for row in data["train"]:
  print(row)
  break

{'id': 'w2CHbo08N9E', 'category': 'Food', 'duration': 518}


In [8]:
import anthropic
import os

GENERATE_QA_PROMPT = """Generate 5 different pairs of questions and answers in JSON format based on the description of the video (in which the description is generated for person without vision can fully understand it).
The questions should be relevant to the video content and the answers should be correct.
Also, diversify the types of questions and answers as much as possible.
Remember to use Vietnamese language to generate the questions and answers.
Examples of questions (do not need to follow the order and these are just examples):
- What's the video about?
- What are key points in the video?
- What is the color of the object?
- What is the person doing?
- What is the person in the video holding and what are its characteristics?
- How does the person in the video look?
- What is the position of the object in the video?
And more questions that can be asked about the video content (what, where, when, why, how, etc.) with varying levels of complexity.
All questions should be relevant to the video content and the answers should be FULLY informative and correct. The answer should be a complete sentence or a complete phrase.
Only return the list of pair of questions and answers in the following JSON format:
[{"question":"Q1","answer":"A1"},{"question":"Q2","answer":"A2"},...]
Your response should be only the JSON list without narrative or additional information.
"""

anthropic_client = anthropic.Anthropic(
  api_key=os.getenv("ANTHROPIC_API_KEY"),
)

message = anthropic_client.messages.create(
  model="claude-3-haiku-20240307",
  system=GENERATE_QA_PROMPT,
  messages=[
    {
      "role": "user",
      "content": "VIDEO CONTENT: Video này có hình ảnh một người phụ nữ mang thai đang mở tủ lạnh. Trong tủ lạnh có rất nhiều trái cây và rau củ. Người phụ nữ đang cầm một củ cà rốt. Bên cạnh tủ lạnh là một người phụ nữ khác đang đeo mặt nạ.",
    },
  ],
  stream=False,
  max_tokens=4096,
  temperature=1,
)

In [12]:
print(message.content[0].text)

[
  {
    "question": "Người phụ nữ đang làm gì trong video?",
    "answer": "Người phụ nữ mang thai đang mở tủ lạnh và cầm một củ cà rốt."
  },
  {
    "question": "Trong tủ lạnh có gì?",
    "answer": "Trong tủ lạnh có rất nhiều trái cây và rau củ."
  },
  {
    "question": "Ai đang đứng bên cạnh tủ lạnh?",
    "answer": "Bên cạnh tủ lạnh là một người phụ nữ khác đang đeo mặt nạ."
  },
  {
    "question": "Người phụ nữ đang cầm vật gì?",
    "answer": "Người phụ nữ đang cầm một củ cà rốt."
  },
  {
    "question": "Người phụ nữ trong video có điều gì đặc biệt?",
    "answer": "Người phụ nữ trong video là người mang thai."
  }
]
